# Car Price prediction

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>


# Отчёт о проделаной работе:
За основу было взято Baseline решение и к нему было сделано ряд добавлений в основном следуя рекомендациям. 



* Очистка данных и их анализ, EDA 
* Извлечение числовых признаков, парсинг признаков, понижение размерности
* Логарифмизация и нормализация признаков (несколько методов)
* Генерация новых признаков.
* Encoding признаков

За счёт вышеописаных удалось значительно поднять метрику по сравнению с Baseline

* Логарифмизация target (сильно помогла ML)
* Были поробованы модели Extra Tree Regressor и XGB Regressor в дополнение к CatBoost

NN:
* Добавление слоёв в нейросеть, перебор параметров активации, LR


NPL:
* Очистка текста: удаление лишних символов 
* Обработка: лемматизация
* Stopword

Images:
* Подбор параметров аугуметации (albumentation)
* Управление LR через ReduceLROnPlateau
* Fine tuning

Assembling:
* Ансамблирование градиентного бустинга и нейронной сети (усреднение их предсказаний)

In [1]:
!pip install -q tensorflow==2.3

In [2]:
#аугментации изображений
!pip install albumentations -q

In [3]:
# Загрузим необходимые библиотеки

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import pandas_profiling
from nltk.corpus import stopwords

# modeling
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

# keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import albumentations as A

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
import seaborn as sns


In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [5]:
# Функция подсчёта метрики MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [7]:
!pip freeze > requirements.txt

# Загрузка данных

In [8]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [9]:
train.info()

In [10]:
train.nunique()

# Machine Learning (ML)
ML объединяет построение регрессий различными методами 

# EDA

## Первичный EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

Посмотрим, как выглядят распределения числовых признаков:

In [11]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Итого:
* CatBoost сможет работать с признаками и в таком виде но лучше проделать EDA

In [12]:
# Сделаем первичный EDA с помощью Pandas profiling
pandas_profiling.ProfileReport(train)

Видим что данные достаточно чистенькие но всё-равно нужно с ними поработать

## Очистка данных и EDA

In [13]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

### Очистка данных

**Пройдёмся по столбцам и почистим их**

*столбец bodyType*

In [14]:
data.bodyType.value_counts(normalize=True)

Удалим лишнюю информацию о дверях которая дублируется в соотвествующем столбце

In [15]:
data.bodyType = data.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
data.bodyType.value_counts(normalize=True)

*столбец brand*

In [16]:
print(data.brand.value_counts(normalize=True))

Тут всё в порядке

*столбец color*

In [17]:
data.color.value_counts()

Тут всё в порядке

столбец engineDisplacement

In [18]:
data.engineDisplacement.value_counts(normalize=True)

Переведём столбец в числовые значения и заполним пропуски наиболее часто встречающимся объемом двигателя

In [19]:
def engineDisplacement_to_float(item):
    try:
        return float(item)
    except:
        return 2.0 # наиболее часто встречающийся объем двигателя

data.engineDisplacement = data.engineDisplacement.apply(lambda x: (x.replace("LTR", "")) if isinstance(x, str) else x)
data.engineDisplacement = data.engineDisplacement.apply(engineDisplacement_to_float)
data.engineDisplacement.value_counts(normalize=True)

*столбец enginePower*

In [20]:
data.enginePower.value_counts(normalize=True)

Переведём столбец в числовые значения и заполним пропуски наиболее часто встречающейся мощьностью двигателя

In [21]:
def enginePower_to_float(item):
    try:
        return float(item)
    except:
        return float(184) # наиболее часто встречающаяся мошьность

data.enginePower = data.enginePower.apply(lambda x: x.split()[0].strip() if isinstance(x, str) else x)
data.enginePower = data.enginePower.apply(enginePower_to_float)
data.enginePower.value_counts(normalize=True)

*столбец fuelType*

In [22]:
data.fuelType.value_counts(normalize=True)

Тут всё в порядке

*столбец modelDate*

In [23]:
data.modelDate.value_counts()

Тут всё в порядке

*столбец model_info*

In [24]:
data.model_info.value_counts()

In [25]:
data.model_info = data.model_info.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
data.model_info.value_counts()

*столбец name*

In [26]:
data.name.value_counts()

Столбец name повторяет столбцы model_info, engineDisplacement, enginePower и Привод. Удалим его.

In [27]:
data.drop(['name'], axis=1, inplace=True)

*столбец numberOfDoors*

In [28]:
data.numberOfDoors.value_counts()

Тут всё в порядке

*столбец productionDate*

In [29]:
data.productionDate.value_counts(normalize=True)

Тут всё в порядке

*столбец vehicleConfiguration*

In [30]:
data.vehicleConfiguration.value_counts()

Столбец complectation_dict повторяет информацию других столбцов. Удалим.

In [31]:
data.drop(['vehicleConfiguration'], axis=1, inplace=True)

*столбец vehicleTransmission*

In [32]:
data.vehicleTransmission.value_counts(normalize=True)

Тут всё в порядке хотя низкий процент автомобилей с механической коробкой выглядет странновато

столбец Владельцы

In [33]:
data.Владельцы.value_counts(normalize=True)

Тут всё в порядке

*столбец Владение*

In [34]:
data.Владение.value_counts(normalize=True)

Выглядет как не очень важный столбец который сильно утяжеляет данные для моделирования, также много пропусков. Удалим.

In [35]:
data.drop(['Владение'], axis=1, inplace=True)

*столбец ПТС*

In [36]:
data.ПТС.value_counts(normalize=True)

Тут всё в порядке

*столбец Привод*

In [37]:
data.Привод.value_counts(normalize=True)

Тут тоже всё в порядке

*столбец Руль*

In [38]:
data.Руль.value_counts(normalize=True)

И тут всё в порядке

**Посмотрим есть ли дубликаты и пропуски**

In [39]:
# Посмотрим на количество дубликатов:
print(sum(data.duplicated()))
print(data.shape)

оказалось что дубликатов нет

In [40]:
# Посмотрим есть ли пропуски
data.info()

Всё хорошо, пропусков нет

## EDA

In [41]:
# Посмотрим количество уникальных значений в каждом столбце
data.nunique(dropna=False)

In [42]:
# Сгруппируем признаки по типам
num_cols = ['engineDisplacement', 'enginePower', 'mileage', 'modelDate', 'productionDate']
bin_cols = ['Руль', 'ПТС']
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 'numberOfDoors',
            'vehicleTransmission', 'Владельцы', 'Привод']
target_cols = ['price']

**Анализ числовых признаков**

In [43]:
# посмотрим на корреляцию числовых признаков
import seaborn as sns
sns.pairplot(data[num_cols])

Видим сильную корреляцию между modelDate и productionDate что ожидаемо. Есть несколько других закомерностей, например, engine power и productionDate.

In [44]:
# Посмотрим на общую информацию
data[num_cols].describe()

In [45]:
# Посмотрим на корреляцию числовых признаков с ценой (нашим таргетом)
plt.figure(figsize=(7, 4));
sns.heatmap(data[data['sample'] == 1][num_cols + ['price']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')

Видим что все признаки сильно влияют на цену за исключением engineDisplacement который в свою очередь тесно связан с enginePower. Также modelDate и productionDate показывают сильную взаимосвязь. Удалим engineDisplacement и modelDate

In [46]:
data.drop(['modelDate', 'engineDisplacement'], axis=1, inplace=True)
for col in ['modelDate', 'engineDisplacement']:
    num_cols.remove(col)

**Анализ бинарных и категоральных признаков**

Посмотрим есть ли дисбаланс между трейновой и тестовой выборками

In [47]:
for col in (bin_cols + cat_cols):
    if col not in ['model_info']:
        fig, ax = plt.subplots(figsize=(7, 2), ncols=2, nrows=1)
        ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 7})
        data[data['sample'] == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
        ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 7})
        data[data['sample'] == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])

Видим что имеется очень хоршее соответсвие.

In [48]:
#Удалим строки автомобилей с правым рулём из трейновой выборки и после этого удалим столбец 'Руль'
#data = data[data['Руль'] == 'Левый']
#data.drop(['Руль'], axis=1, inplace=True)
#print(bin_cols)
#bin_cols.remove('Руль')
#bin_cols

**Анализ таргета (price)**

In [49]:
# посмотрим на таргет (price)
data.query('sample == 1').price.hist();
plt.title('The target variable distribution', fontdict={'fontsize': 7});
plt.xlabel('price, RUB * 10^7')

Большой хвост по цене, поэтому прологорифмируем наш таргет и добавим его как новый столбец

In [50]:
np.log2(data.query('sample == 1').price).hist();
plt.title('The log2 target variable distribution', fontdict={'fontsize': 7});
data['price_log2'] = np.log2(data.price + 1)

### Feature Engineering

In [51]:
print(num_cols)

Среди трёх числовых признаков лишь productionDate вызывает ворос. Лучше использовать возраст автомобиля вместо года производства. Также создадим новый признак: средний годовой пробег.

In [52]:
data['age'] = 2021 - data.productionDate
data['mileage_per_year'] = round(data['mileage'] / data['age'], 0)
num_cols = num_cols+['age','mileage_per_year']
num_cols.remove('productionDate')
data.drop(['productionDate'], axis=1, inplace=True)

Также создадим следующие новые признаки:

1) столбец обозначающий старые автомобили (старше 10 лет) так как цена на них не сильно зависит от возраста

2) столбец обозначающий редкие цвета так как это часто дорогие автомобили

3) автомобили с редкими типами кузова которые могут обозначать высокую цену

Признаки эти бинарны

In [53]:
data.bodyType.value_counts(normalize=True)

In [54]:
data['old_car'] = data.age.apply(lambda x: 1 if x >10 else 0)
data['rare_colors'] = data.color.apply(lambda x: 1 if x in ['фиолетовый', 'пурпурный', 'золотистый', 'оранжевый', 'жёлтый', 'розовый'] else 0)
data['rare_bodyType'] = data.bodyType.apply(lambda x: 1 if x in ['кабриолет', 'компактвэн', 'лимузин', 'пикап'] else 0)
bin_cols = bin_cols+['old_car','rare_colors','rare_bodyType']

In [55]:
# Прологорифмируем числовые признаки
data['enginePower_log2'] = np.log2(data.enginePower+1)
data['enginePower_log2'].replace([np.inf, -np.inf], 0, inplace=True)
data['mileage_log2'] = np.log2(data.mileage+1)
data['mileage_log2'].replace([np.inf, -np.inf], 0, inplace=True)
data['age_log2'] = np.log2(data.age+1)
data['age_log2'].replace([np.inf, -np.inf], 0, inplace=True)
data['mileage_per_year_log2'] = np.log2(data.mileage_per_year+1)
data['mileage_per_year_log2'].replace([np.inf, -np.inf], 0, inplace=True)
# Удалим старые слолбцы
data.drop(['enginePower','mileage','age','mileage_per_year'], axis=1, inplace=True)
num_cols = ['enginePower_log2', 'mileage_log2', 'age_log2', 'mileage_per_year_log2']

In [56]:
# Нормализация числовых признаков (несколько методов было попробывано):
scaler = StandardScaler()
#scaler = RobustScaler()
#scaler = MinMaxScaler()
for column in num_cols:
        data[column] = scaler.fit_transform(data[[column]])[:,0]

In [57]:
# Посмотрим на корреляцию новых числовых параметров с ценой (нашим таргетом)
plt.figure(figsize=(7, 4));
sns.heatmap(data[data['sample'] == 1][num_cols + ['price_log2']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')

In [58]:
# Сделаем Encoding для бинарных и категоральных признаков

for colum in (bin_cols + cat_cols):
    data[colum] = data[colum].astype('category').cat.codes

#Label encoding:    
#cols_to_encode = list(set(data.columns) & set(cat_cols))
#for colum in cols_to_encode:
#    data[colum] = data[colum].astype('category').cat.codes

# One-Hot Encoding: в pandas есть готовая функция - get_dummies.
data = pd.get_dummies(data, columns=(bin_cols + cat_cols), dummy_na=False)

Выяснилось что One-Hot справляется лучше для подачи в нейросети 

In [59]:
# Теперь посмотрим на важность бинарных и категоральных признаков
#from sklearn.feature_selection import mutual_info_regression
#imp_cat = pd.Series(
#    mutual_info_regression(
#        data[data['sample'] == 1][list(set(data.columns) & set(cat_cols+bin_cols))], 
#        data[data['sample'] == 1]['price_log2'], 
#        discrete_features=True), index=list(set(data.columns) & set(cat_cols+bin_cols))
#)
#imp_cat.sort_values(inplace=True)
#imp_cat.plot(kind='barh', title='Важность категоральных и бинарных признаков')
#plt.show()

В принципе всё пристойно и можно подавать в модели

## Model 1: Наивная модель
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [60]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [61]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

## Model 2: CatBoostRegressor

In [62]:
X = data.query('sample == 1').drop(['sell_id', 'price', 'price_log2', 'sample', 'description'], axis=1)
X_sub = data.query('sample == 0').drop(['sell_id', 'price','price_log2', 'sample', 'description'], axis=1)
y = data.query('sample == 1').price

In [63]:
# Сделаем разбиение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

Возьмём таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).

In [64]:
catboost_log = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
catboost_log.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

predict_catboost_log = np.exp(catboost_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_catboost_log))*100:0.2f}%")

## Model 3: Extra Tree Regressor

In [65]:
# Запустим модель со стандартными параметрами
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))
predict_etr_log = np.exp(etr_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_etr_log))*100:0.2f}%")

## Model 4: XGB Regressor

In [66]:
# Запустим модель с параметрами подобранными вручную
import xgboost as xgb
xgb_log = xgb.XGBRegressor(
    objective='reg:squarederror', 
    colsample_bytree = 0.4,               
    learning_rate=0.125, 
    max_depth=12, 
    alpha=1,                   
    n_estimators=1000,
    random_state=RANDOM_SEED,
    verbose=1, 
    n_jobs=-1)

xgb_log.fit(X_train, np.log(y_train))
predict_xgb_log = np.exp(xgb_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_xgb_log))*100:0.2f}%")

### Submission

In [67]:
VERSION = 4
submission_catboost_log = np.exp(catboost_log.predict(X_sub))
sample_submission['price'] = submission_catboost_log
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

## Model 5: Tabular NN

In [68]:
# Уберём малозначимые столбцы (исходя из диаграммы значимости, см. выше)
#X_train.drop(['Руль', 'rare_colors', 'rare_bodyType', 'mileage_per_year_log2'], axis=1, inplace=True)
#X_test.drop(['Руль', 'rare_colors', 'rare_bodyType', 'mileage_per_year_log2'], axis=1, inplace=True)
#X_sub.drop(['Руль', 'rare_colors', 'rare_bodyType', 'mileage_per_year_log2'], axis=1, inplace=True)

В итоге было решено оставить все признаки

In [69]:
tf.keras.backend.clear_session()

Построим обычную сеть:

In [70]:
# Base model
#model = Sequential()
#model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
#model.add(L.Dropout(0.5))
#model.add(L.Dense(256, activation="relu"))
#model.add(L.Dropout(0.5))
#model.add(L.Dense(1, activation="linear"))

In [71]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid")) #замена relu -> sigmoid
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dense(256, activation="relu")) #добавили допонительный полносвязный слой
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [72]:
# Более сложная модель
#from tensorflow.keras import regularizers
#model = Sequential()
#model.add(L.Dense(512, input_dim=X_train.shape[1], 
#                  activation="sigmoid",
##                   activation='relu',
#                  kernel_regularizer=regularizers.l2(l2=1e-6),
#                  bias_regularizer=regularizers.l2(l2=1e-6),
#                 ))
#model.add(L.Dropout(0.5))
#model.add(L.Dense(256, kernel_regularizer=regularizers.l2(l2=1e-5),
#                  bias_regularizer=regularizers.l2(l2=1e-5),
#                  activation="relu"))
#model.add(L.Dropout(0.5))
#model.add(L.Dense(256, kernel_regularizer=regularizers.l2(l2=0.00001), 
#                  bias_regularizer=regularizers.l2(l2=0.00001),
#                  activation="relu"))
#model.add(L.Dropout(0.5))
#model.add(L.Dense(32, kernel_regularizer=regularizers.l2(l2=0.001),
#                  bias_regularizer=regularizers.l2(l2=0.001),
#                  activation="relu"))
## model.add(L.Dropout(0.25))
#model.add(L.Dense(1, activation="linear"))

Более сложная модель однако не дала ощутимого прироста

In [73]:
model.summary()

In [74]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

Были попытки поменять LR которые однако не улучшили результат

In [75]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [76]:
history = model.fit(X_train, y_train,
                    batch_size=524,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [77]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [78]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [79]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [80]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:, 0]
sample_submission.to_csv('nn1_submission.csv', index=False)

Примечательно что нейросеть таки побила ML методы

## Model 6: NLP + Multiple Inputs

In [81]:
data.description

In [82]:
!pip install pymorphy2
!pip install pymorphy2-dicts
import pymorphy2

In [83]:
morphy = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

In [84]:
#создадим функцию. в которой будет и лемматизация и очистка текста
#tras_sym = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
trash_sym = "[^А-я ]+" #Более точная очистка, которая оставляет только слова


def lemma(text):
    text = text.lower()  # понижаем регистр
    text = re.sub(trash_sym, ' ', text)  # удаляем символы из паттерна
    strings = []  # создаем массив, в котором будут храниться лемматизированные строки
    for wrd in text.split():  # берем слово из строки
        wrd = wrd.strip()  # убираем пробелы до и после слова
        wrd = morphy.normal_forms(wrd)[0]  # приводим к нормальной форме
        strings.append(wrd)  # добавляем слово в строку массива
    return ' '.join(strings)  # вернем значения, разделив пробелами

In [85]:
strings_set = []
strings_set = df_NLP.apply(
    lambda df_NLP: lemma(df_NLP.description), axis=1)

In [86]:
russian_stopwords = stopwords.words("russian")

In [87]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]


In [88]:
# применим функцию к нашим лемматизированым строкам слов
str_without_stop = [lineWithoutStopWords(line) for line in strings_set]

In [89]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [90]:
data.info()

In [91]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

### Tokenizer

In [92]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

In [93]:
tokenize.word_index

In [94]:
%%time
text_train_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape,
      text_test_sequences.shape, text_sub_sequences.shape, )

In [95]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

### RNN NLP

In [96]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

In [97]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="sigmoid")) #замена relu -> sigmoid
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dense(256, activation="relu")) #добавили допонительный полносвязный слой
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(1, activation="linear"))

### Multiple Inputs NN

In [98]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [99]:
model.summary()

### Fit

In [100]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [101]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [102]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [103]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [104]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [105]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

Результат немрого просел при добавлении NLP к NN несмотря на преобразования текста

In [106]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

## Model 7: Добавляем картинки

### Data

In [107]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (6,4))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [108]:
size = (400, 300)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [109]:
IMG_SIZE = [size[1],size[0]]

In [110]:
import albumentations as A
AUGMENTATIONS_TRAIN = A.Compose([ #Для тренировки нейросети
    A.GaussianBlur(p=0.05),
    A.ShiftScaleRotate(shift_limit=0.08, 
                       scale_limit=0.05, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=15, 
                       p=.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, 
                                                contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, 
                                                contrast_limit=0.1)],
        p=0.5),
    A.HorizontalFlip(p=0.4),
 #   A.HueSaturationValue(p=0.4),
    A.RGBShift(p=0.4),
    A.FancyPCA(alpha=0.1, 
                always_apply=False, p=0.4),
    A.Resize(IMG_SIZE[0],IMG_SIZE[1]),
  #  A.ToFloat(max_value = 255)
])


# Выведем пример аугментации
plt.figure(figsize=(12, 8))
for i in range(9):
    img = AUGMENTATIONS_TRAIN(image=images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [111]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [112]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

In [113]:
# аугментация изображений
def process_image(image):
    return AUGMENTATIONS_TRAIN(image=image.numpy())['image']

# векторизация строки
def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen=MAX_SEQUENCE_LENGTH)

# применение векторизации к тексту
def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

# функция  для применения вышеупомянутых функций к входным данным обучающей выборки
def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image, ] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

# функция  для применения вышеупомянутых функций к входным данным валидационной и тестовой выборкок
def tf_process_val_dataset_element(image, table_data, text, price):
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

# использование tf.data.Dataset с использованием функций для обучающей выборки
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
)).map(tf_process_train_dataset_element)
# использование tf.data.Dataset с использованием функций для валидационной выборки
test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
)).map(tf_process_val_dataset_element)
# использование tf.data.Dataset с использованием функций для тестовой выборки
y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
)).map(tf_process_val_dataset_element)

# проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__()
test_dataset.__iter__().__next__()
sub_dataset.__iter__().__next__()

### Строим сверточную сеть для анализа изображений без "головы"

In [114]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

**Добавим fine tuning**

In [115]:
efficientnet_model.trainable = False

In [116]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [117]:
#строим нейросеть для анализа табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    ])

In [118]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [119]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
model.summary()

In [120]:
# укажем используемый оптимизатор и начальную скорость обучения
optimizer = tf.keras.optimizers.Adam(2e-3)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

Добавим управление LR через ReduceLROnPlateau

In [121]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=10, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=3,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=0.00001,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

In [122]:
history = model.fit(train_dataset.batch(30),
                    epochs=48,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

In [123]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [124]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

Метрика немного улучшилась по сравнению с тем что было до добавления картинок

In [125]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

Разморозим половину слоёв

In [126]:
efficientnet_model.trainable = True
nlp_model.trainable = True
tabular_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = round(len(efficientnet_model.layers)/2)

# Freeze all the layers before the `fine_tune_at` layer
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [127]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=5, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=2,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=1e-7,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

In [128]:
# укажем используемый оптимизатор и начальную скорость обучения (ниже чем было ранее)
optimizer = tf.keras.optimizers.Adam(1e-5)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [129]:
model.summary()

In [130]:
history = model.fit(train_dataset.batch(30),
                    epochs=12,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

In [131]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [132]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

Разморозим оставшиеся слои

In [133]:
efficientnet_model.trainable = True
nlp_model.trainable = True
tabular_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = round(len(efficientnet_model.layers)/2)

# Freeze all the layers before the `fine_tune_at` layer
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  True

In [134]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=5, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=2,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=1e-7,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

In [135]:
# укажем используемый оптимизатор и начальную скорость обучения (ниже чем было ранее)
optimizer = tf.keras.optimizers.Adam(5e-6)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [136]:
model.summary()

In [137]:
history = model.fit(train_dataset.batch(30),
                    epochs=12,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

In [138]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [139]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [140]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:, 0]
sample_submission.to_csv('nn3_submission.csv', index=False)

# Blend

In [141]:
blend_predict = (predict_catboost_log + test_predict_nn3[:,0])/2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [142]:
blend_sub_predict = (submission_catboost_log + sub_predict_nn3[:,0])/2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)